In [1]:
import numpy as np

In [2]:
class AffineTransformationLayer:
    def __init__(self, input_size, output_size, weight_initializer, random_init_scaler=True):
        self.input_size = input_size
        self.output_size = output_size
        if weight_initializer == 'zeros':
            self.weights = np.zeros((input_size, output_size))
            self.bias = np.zeros((1, output_size))
        elif weight_initializer == 'random_init':
            self.weights = np.random.random((input_size, output_size))
            self.bias = np.random.random((1, output_size))
            if random_init_scaler:
                self.weights = self.weights / np.sqrt(input_size + output_size)
                self.bias = self.bias / np.sqrt(input_size + output_size)
        elif weight_initializer == 'normal_init':
            self.weights = np.random.normal(0, 0.1, (input_size, output_size))
            self.bias = np.random.normal(0, 0.1, (1, output_size))

    def forward(self, input):
        self.input = input
        return np.dot(input, self.weights) + self.bias

    def backward(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [3]:
class ActivationLayer:
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime
    
    def forward(self, input):
        self.input = input
        return self.activation(input)
    
    def backward(self, output_error, learning_rate):
        return output_error * self.activation_prime(self.input)

In [4]:
# bonus
class FlattenLayer:
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def forward(self, input):
        return np.reshape(input, (1, -1))
    
    def backward(self, output_error, learning_rate):
        return np.reshape(output_error, self.input_shape)

In [5]:
# bonus
class SoftmaxLayer:
    def __init__(self, input_size):
        self.input_size = input_size
    
    def forward(self, input):
        self.input = input
        tmp = np.exp(input)
        self.output = tmp / np.sum(tmp)
        return self.output
    
    def backward(self, output_error, learning_rate):
        input_error = np.zeros(output_error.shape)
        out = np.tile(self.output.T, self.input_size)
        return self.output * np.dot(output_error, np.identity(self.input_size) - out)

In [6]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return np.exp(-x) / (1 + np.exp(-x))**2

def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1 - np.tanh(x)**2

def relu(x):
    return np.maximum(x, 0)

def relu_prime(x):
    return np.array(x >= 0).astype('int')

def leaky_relu(x):
    return np.maximum(x, 0.01 * x)

def leaky_relu_prime(x):
    return np.array(x >= 0).astype('int') + 0.01 * np.array(x < 0).astype('int')

def linear(x):
    return x

def linear_prime(x):
    return np.ones(x.shape)

In [7]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / y_pred.size

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def mae_prime(y_true, y_pred):
    return np.sign(y_pred - y_true) / y_pred.size

def cross_entropy(y_true, y_pred):
    return -np.sum(y_true * np.log(y_pred))

def cross_entropy_prime(y_true, y_pred):
    return -y_true / y_pred

In [8]:
from utils import *
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

x_train, y_train, x_test, y_test = load_mnist()

# Normalize the images.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train = x_train[:10000]
y_train = y_train[:10000]

# Flatten the images.
x_train = x_train.reshape((-1, 784))
x_test = x_test.reshape((-1, 784))

# One hot encode the labels.
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Split the train data into train and validation sets.
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

In [9]:
class NeuralNetwork:
    def __init__(self, n_inputs,  input_shape, hidden_layer_sizes, n_outs, activation_fn, activation_fn_derivative, weight_initialization, loss, loss_prime):
        self.network = []
        self.network.append(FlattenLayer(input_shape))
        # Input To Hidden
        self.network.append(AffineTransformationLayer(n_inputs, hidden_layer_sizes[0], weight_initialization))
        self.network.append(ActivationLayer(activation_fn, activation_fn_derivative))
        # Hidden To Hidden
        for i in range(len(hidden_layer_sizes) - 1):
            self.network.append(AffineTransformationLayer(hidden_layer_sizes[i], hidden_layer_sizes[i+1], weight_initialization))
            self.network.append(ActivationLayer(activation_fn, activation_fn_derivative))
        # Hidden To Output
        self.network.append(AffineTransformationLayer(hidden_layer_sizes[-1], n_outs, weight_initialization))
        self.loss = loss
        self.loss_prime = loss_prime
        self.train_loss = []
        self.val_loss = []
    
    def predict(self, x):
        for layer in self.network:
            x = layer.forward(x)
        return x
    
    def fit(self, x_train, y_train, x_valid, y_valid, epochs=1000, learning_rate=0.1, batch_size=32, early_stopping=True, patience=10, verbose=True):
        x_b = np.array_split(x_train, len(x_train)/ batch_size)
        y_b = np.array_split(y_train, len(y_train)/ batch_size)
        for epoch in range(epochs):
            error = 0
            error_val = 0
            for x_train, y_train in zip(x_b,y_b):
                for x, y_true in zip(x_train, y_train):

                    output = x
                    
                    output = self.forward(output)

                    error += self.loss(y_true, output)

                    output_error = self.loss_prime(y_true, output)

                    self.backward(output_error, learning_rate)

                    error /= len(x_train)

            error_val = sum([self.loss(y, self.predict(x)) for x, y in zip(x_valid, y_valid)]) / len(x_valid) 

            self.train_loss.append(error)
            self.val_loss.append(error_val)

            if early_stopping:
                if epoch > patience:
                    if self.train_loss[-1] > np.mean(self.val_loss[-(patience+1):-1]):
                        print('Early Stopping')
                        break
            if verbose:
                if epoch % 10 == 0:
                    print(f"Epoch: {epoch}, Training Loss: {error}, Validation Loss: {error_val}")

    def forward(self, x):
        for layer in self.network:
            x = layer.forward(x)
        return x

    def backward(self, output_error, learning_rate):
        for layer in reversed(self.network):
            output_error = layer.backward(output_error, learning_rate)

    def predict_proba(self, x):
        return self.predict(x)

    def score(self, x, y):
        cnt = 0
        for x_v, y_v in zip(x,y):
            op = self.predict(x_v)
            if (np.argmax(op) == np.argmax(y_v)):
                cnt+=1
        return cnt/y.shape[0]

In [10]:
def save_model(model, filename):
    import pickle
    with open(f"Model_Saves/{filename}", 'wb') as f:
        pickle.dump(model, f)

def load_model(filename):
    import pickle
    with open(f"Model_Saves/{filename}", 'rb') as f:
        return pickle.load(f)

In [11]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((8000, 784), (8000, 10), (2000, 784), (2000, 10))

In [12]:
activation_fns = [relu, sigmoid, tanh, leaky_relu, linear]
activation_fns_prime = [relu_prime, sigmoid_prime, tanh_prime, leaky_relu_prime, linear_prime]

In [14]:
for activation_fn, activation_fn_prime in zip(activation_fns, activation_fns_prime):
    print(f"Activation Function: {activation_fn.__name__}")
    model = NeuralNetwork(784, (28,28), [256,128,64,32], 10, activation_fn, activation_fn_prime, 'normal_init', mse, mse_prime)
    model.fit(x_train, y_train, x_val, y_val, epochs=50, learning_rate=0.001, batch_size=128, early_stopping=True, patience=10, verbose=True)
    print(f"Accuracy: {model.score(x_test, y_test)}")
    save_model(model, f"model_{activation_fn.__name__}.pkl")

Activation Function: relu
Epoch: 0, Training Loss: 0.0007984265796086321, Validation Loss: 0.07986270092955512


In [ ]:
initialization_fns = ['zeros', 'normal_init', 'random_init']

In [ ]:
for initialization_fn in initialization_fns:
    print(f"Initialization Function: {initialization_fn}")
    model = NeuralNetwork(784, (28,28), [256,128,64,32], 10, relu, relu_prime, initialization_fn, mse, mse_prime)
    model.fit(x_train, y_train, x_val, y_val, epochs=50, learning_rate=0.001, batch_size=128, early_stopping=True, patience=10, verbose=True)
    print(f"Accuracy: {model.score(x_test, y_test)}")
    save_model(model, f"model_{initialization_fn}_activation_relu.pkl")

Epoch: 0, Training Loss: 0.0022557183192438964, Validation Loss: 0.08022426883322133
Epoch: 1, Training Loss: 0.0018364025694024208, Validation Loss: 0.07063569194875897
Epoch: 2, Training Loss: 0.0015514003335942873, Validation Loss: 0.06413582115634027
Epoch: 3, Training Loss: 0.001321268683615486, Validation Loss: 0.05958066708476712
Epoch: 4, Training Loss: 0.0011662841823164183, Validation Loss: 0.05586707933478441
Epoch: 5, Training Loss: 0.0010235125091393783, Validation Loss: 0.05259582749877764
Epoch: 6, Training Loss: 0.0009198648657909622, Validation Loss: 0.04968066652503007
Epoch: 7, Training Loss: 0.0008633215877789182, Validation Loss: 0.047105525658676116
Epoch: 8, Training Loss: 0.0008442904916260939, Validation Loss: 0.0448600442300863
Epoch: 9, Training Loss: 0.0008036938675641232, Validation Loss: 0.04287671746424573
Epoch: 10, Training Loss: 0.0007692231982143351, Validation Loss: 0.04111311313056513
Epoch: 11, Training Loss: 0.0007323039390737545, Validation Loss:

In [ ]:
print('Accuracy: %f' % nn.score(x_test, y_test))

Accuracy: 0.933100


In [ ]:
# Plot the loss.
plt.plot(nn.train_loss, label='train loss')
plt.plot(nn.val_loss, label='validation loss')
plt.legend()
plt.show()